<a href="https://colab.research.google.com/github/zachfreitas/ADS_505_Applied_Data_Sci_for_Business-/blob/main/Module_5/Assignment_5/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 5
Zachariah Freitas<br>
2022-10-04

Install Libraries

In [1]:
# Connect Colab to Google Drive for Convience.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [51]:
# Install Book Libraries
!pip install dmba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
# Import Libraries

# EDA and Preprep
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import numpy as np

from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

# Classification Model Libraries
# from sklearn.linear_model import LogisticRegressionCV # lr or Logistic Regression
# from sklearn.ensemble import BaggingClassifier # bc or Bagging Classifier
# from sklearn.ensemble import AdaBoostClassifier # abc or Ada Boost Classifier
# from sklearn.ensemble import RandomForestClassifier # rf or Random Forest Classifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis # lda or Linear Discriminant Analysis
# from sklearn.neural_network import MLPClassifier # nn or Neural Nets
# from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import pairwise
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans


# Book Helper Functions
from dmba import plotDecisionTree, gainsChart, liftChart, classificationSummary, regressionSummary
from dmba.metric import AIC_score
# import dmba

# Plotting Libraries
import matplotlib.pylab as plt
import seaborn as sns
from pandas.plotting import parallel_coordinates

# Inline Plot Formating
%matplotlib inline

In [53]:
# Dataset Sources
dataSource = 'https://raw.githubusercontent.com/zachfreitas/ADS_505_Applied_Data_Sci_for_Business-/main/Module_5/Assignment_5/BathSoapHousehold.csv'

In [54]:
df = pd.read_csv(dataSource)
df_copy = df.copy()

df_copy.columns = [d.replace(' ', '_').replace('.', '') for d in df_copy.columns]
# df_copy = df_copy.iloc[: , 2:]
# Drop unnecessary columns
# df_copy = df_copy.drop('TARGET_D', axis=1)
df_copy.head()

,Member_id,SEC,FEH,MT,SEX,AGE,EDU,HS,CHILD,CS,...,PropCat_6,PropCat_7,PropCat_8,PropCat_9,PropCat_10,PropCat_11,PropCat_12,PropCat_13,PropCat_14,PropCat_15
0,1010010,4,3,10,1,4,4,2,4,1,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.028037,0.0,0.130841,0.339564
1,1010020,3,2,10,2,2,4,4,2,1,...,0.347048,0.026834,0.016100,0.014311,0.0,0.059034,0.000000,0.0,0.080501,0.000000
2,1014020,2,3,10,2,4,5,6,4,1,...,0.121212,0.033550,0.010823,0.008658,0.0,0.000000,0.016234,0.0,0.561688,0.003247
3,1014030,4,0,0,0,4,0,0,5,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.600000,0.000000
4,1014190,4,1,10,2,3,4,4,3,1,...,0.000000,0.000000,0.048193,0.000000,0.0,0.000000,0.000000,0.0,0.144578,0.000000


In [55]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 46 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Member_id              600 non-null    int64  
 1   SEC                    600 non-null    int64  
 2   FEH                    600 non-null    int64  
 3   MT                     600 non-null    int64  
 4   SEX                    600 non-null    int64  
 5   AGE                    600 non-null    int64  
 6   EDU                    600 non-null    int64  
 7   HS                     600 non-null    int64  
 8   CHILD                  600 non-null    int64  
 9   CS                     600 non-null    int64  
 10  Affluence_Index        600 non-null    int64  
 11  No_of_Brands           600 non-null    int64  
 12  Brand_Runs             600 non-null    int64  
 13  Total_Volume           600 non-null    int64  
 14  No_of__Trans           600 non-null    int64  
 15  Value 

In [56]:
n=10
N=15
df_copy.iloc[: , n:-N].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Affluence_Index        600 non-null    int64  
 1   No_of_Brands           600 non-null    int64  
 2   Brand_Runs             600 non-null    int64  
 3   Total_Volume           600 non-null    int64  
 4   No_of__Trans           600 non-null    int64  
 5   Value                  600 non-null    float64
 6   Trans_/_Brand_Runs     600 non-null    float64
 7   Vol/Tran               600 non-null    float64
 8   Avg_Price_             600 non-null    float64
 9   Pur_Vol_No_Promo_-_%   600 non-null    float64
 10  Pur_Vol_Promo_6_%      600 non-null    float64
 11  Pur_Vol_Other_Promo_%  600 non-null    float64
 12  Br_Cd_57,_144          600 non-null    float64
 13  Br_Cd_55               600 non-null    float64
 14  Br_Cd_272              600 non-null    float64
 15  Br_Cd_

In [57]:
df_copy.iloc[: , n:-N].columns

Index(['Affluence_Index', 'No_of_Brands', 'Brand_Runs', 'Total_Volume',
       'No_of__Trans', 'Value', 'Trans_/_Brand_Runs', 'Vol/Tran', 'Avg_Price_',
       'Pur_Vol_No_Promo_-_%', 'Pur_Vol_Promo_6_%', 'Pur_Vol_Other_Promo_%',
       'Br_Cd_57,_144', 'Br_Cd_55', 'Br_Cd_272', 'Br_Cd_286', 'Br_Cd_24',
       'Br_Cd_481', 'Br_Cd_352', 'Br_Cd_5', 'Others_999'],
      dtype='object')

In [58]:
# Demographics that we want to ignore
Demographics = ['SEC', 'FEH' , 'MT', 'SEX', 'AGE', 'EDU', 'HS', 'SHILD', 'CS', 'Affluence_Index']

# Purchase behavior (volume, frequency, susceptibility to discounts, and brand loyalty)
Purchase_behavior = ['No_of_Brands', 'Brand_Runs', 'Total_Volume', 'No_of__Trans', 'Value',
                     'Trans_/_Brand_Runs', 'Vol/Tran', 'Avg_Price_', 'Pur_Vol_No_Promo_-_%',
                     'Pur_Vol_Promo_6_%', 'Pur_Vol_Other_Promo_%', 'Br_Cd_57,_144',
                     'Br_Cd_55', 'Br_Cd_272', 'Br_Cd_286', 'Br_Cd_24', 'Br_Cd_481',
                     'Br_Cd_352', 'Br_Cd_5', 'Others_999'] 

# Basis of purchase (price, selling proposition)
Basis_of_purchase = ['Pr_Cat_1', 'Pr_Cat_2', 'Pr_Cat_3', 'Pr_Cat_4', 
                       'PropCat_5', 'PropCat_6', 'PropCat_7','PropCat_8',
                       'PropCat_9', 'PropCat_10', 'PropCat_11', 'PropCat_12',
                       'PropCat_13','PropCat_14','PropCat_15']  

## Question 1
Use k-means clustering to identify clusters of households based on:


In [17]:
utilities_df = df_copy
# set row names to the utilities column
utilities_df.set_index('Member_id', inplace=True)
# while not required, the conversion of integer data to float
# will avoid a warning when applying the scale function
utilities_df = utilities_df.apply(lambda x: x.astype('float64'))
# compute Euclidean distance
d = pairwise.pairwise_distances(utilities_df, metric='euclidean')
pd.DataFrame(d, columns=utilities_df.index, index=utilities_df.index)

Member_id,1010010,1010020,1014020,1014030,1014190,1017020,1017110,1017160,1017360,1017460,...,1166470,1166740,1166870,1166960,1166980,1167090,1167230,1167340,1167350,1167670
Member_id,,,,,,,,,,,,,,,,,,,,,
1010010,0.000000,6012.402942,15117.555482,6563.048451,468.985234,10189.333866,1934.934462,1299.583187,18618.380315,600.954526,...,1931.142124,1125.079293,566.355089,5253.492447,2732.632046,1079.864648,2979.307699,21237.186854,2550.727318,4346.678033
1010020,6012.402942,0.000000,9129.005098,12573.210463,5786.154789,4201.124657,4081.219769,4805.580576,12612.505608,6556.899264,...,7929.068358,7091.292598,5738.412908,11262.999477,8734.507223,4972.102507,8965.166014,15233.452140,8562.820112,1781.725624
1014020,15117.555482,9129.005098,0.000000,21678.013169,14864.871466,4931.733769,13183.816416,13868.981453,3636.667804,15674.561471,...,17041.788682,16207.424913,14864.152814,20368.973764,17846.431199,14094.188750,18084.428322,6202.225897,17663.197646,10827.027839
1014030,6563.048451,12573.210463,21678.013169,0.000000,6821.823315,16750.980693,8497.165148,7813.329418,25176.686424,6019.324460,...,4650.543268,5490.028181,6856.942152,1318.614080,3846.081895,7605.685495,3631.196312,27794.817930,4015.948182,10886.209558
1014190,468.985234,5786.154789,14864.871466,6821.823315,0.000000,9939.669611,1720.960846,1035.240738,18367.276078,961.308404,...,2252.508809,1422.974419,741.794411,5523.588367,3034.733063,985.399373,3292.558119,20982.616719,2824.751873,4085.113805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167090,1079.864648,4972.102507,14094.188750,7605.685495,985.399373,9163.083166,949.169136,705.658275,17576.835451,1586.468182,...,2967.166633,2121.957804,784.673118,6297.548165,3770.660414,0.000000,3998.592430,20197.036179,3607.630129,3312.545153
1167230,2979.307699,8965.166014,18084.428322,3631.196312,3292.558119,13155.433436,4906.716333,4263.763014,21573.919831,2420.522966,...,1053.149054,1910.027705,3247.916521,2321.691247,305.243925,3998.592430,0.000000,24194.658806,606.329796,7303.135258
1167340,21237.186854,15233.452140,6202.225897,27794.817930,20982.616719,11064.657102,19302.881356,20003.457589,2623.260867,21781.072125,...,23159.396319,22309.421531,20955.539047,26489.327653,23964.297000,20197.036179,24194.658806,0.000000,23786.684648,16911.520340


In [60]:
# scikit-learn uses population standard deviation
utilities_df_norm = utilities_df.apply(preprocessing.scale, axis=0)
# pandas uses sample standard deviation
utilities_df_norm = (utilities_df - utilities_df.mean())/utilities_df.std()
# compute normalized distance based on Sales and Fuel Cost
utilities_df_norm[Basis_of_purchase]
d_norm = pairwise.pairwise_distances(utilities_df_norm[Basis_of_purchase],
                                     metric='euclidean')
pd.DataFrame(d_norm, columns=utilities_df.index, index=utilities_df.index)
 

Member_id,1010010,1010020,1014020,1014030,1014190,1017020,1017110,1017160,1017360,1017460,...,1166470,1166740,1166870,1166960,1166980,1167090,1167230,1167340,1167350,1167670
Member_id,,,,,,,,,,,,,,,,,,,,,
1010010,0.000000,4.591629,4.723974,4.852905,5.933101,2.166992,3.280898,5.872851,4.629737,5.213741,...,4.305512,5.049416,7.863918,2.413057,4.296766,10.181521,4.917289,4.188556e+00,4.211602,4.551483
1010020,4.591629,0.000000,3.222206,3.690428,4.982683,3.773216,3.007222,5.322785,1.775872,2.361817,...,3.132390,3.366208,6.969585,5.616261,1.872141,9.006437,2.255421,1.837187e+00,1.809627,3.140682
1014020,4.723974,3.222206,0.000000,1.225094,5.266154,4.517458,3.960901,2.259828,3.961890,3.854871,...,3.941951,4.417470,7.370281,6.136166,3.740323,9.865780,4.184776,2.715261e+00,2.787416,4.363476
1014030,4.852905,3.690428,1.225094,0.000000,5.155895,4.666277,3.738412,2.267945,4.069626,4.648651,...,4.204461,4.621550,7.435041,6.110871,3.759532,9.868515,4.877441,2.705879e+00,2.791074,4.051575
1014190,5.933101,4.982683,5.266154,5.155895,0.000000,4.749252,3.840500,6.202053,5.376104,5.263533,...,5.305137,5.545413,8.371517,6.423355,4.791889,10.732826,5.632112,4.801271e+00,3.934835,5.140879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167090,10.181521,9.006437,9.865780,9.868515,10.732826,10.043683,9.613789,10.687789,9.490447,10.212460,...,9.683912,9.044241,10.870159,10.496892,9.188658,0.000000,9.936855,9.415927e+00,9.552152,9.577205
1167230,4.917289,2.255421,4.184776,4.877441,5.632112,4.123790,4.190396,5.998977,3.422794,1.936904,...,4.049033,3.606198,6.699972,5.675325,3.370141,9.936855,0.000000,3.318722e+00,3.304222,4.670616
1167340,4.188556,1.837187,2.715261,2.705879,4.801271,3.744988,2.173049,4.750528,1.683550,3.608571,...,2.617042,3.263900,6.671154,5.300337,1.222229,9.415927,3.318722,2.107342e-08,1.125872,1.984889


In [ ]:
# Code for k-means

# Load and preprocess data
utilities_df = pd.read_csv('Utilities.csv')
utilities_df.set_index('Company', inplace=True)
utilities_df = utilities_df.apply(lambda x: x.astype('float64'))
# Normalize distances
utilities_df_norm = utilities_df.apply(preprocessing.scale, axis=0)
kmeans = KMeans(n_clusters=6, random_state=0).fit(utilities_df_norm)
# Cluster membership
memb = pd.Series(kmeans.labels_, index=utilities_df_norm.index)
for key, item in memb.groupby(memb):
     print(key, ': ', ', '.join(item.index))


In [ ]:
# Centroids
centroids = pd.DataFrame(kmeans.cluster_centers_, columns=utilities_df_norm.columns)
pd.set_option('precision', 3)
centroids

In [ ]:
# Within-cluster sum of squared distances and cluster count

# calculate the distances of each data point to the cluster centers
distances = kmeans.transform(utilities_df_norm)
# find closest cluster for each data point
minSquaredDistances = distances.min(axis=1) ** 2
# combine with cluster labels into a data frame
df = pd.DataFrame({'squaredDistance': minSquaredDistances, 'cluster': kmeans.labels_},
    index=utilities_df_norm.index)
# group by cluster and print information
for cluster, data in df.groupby('cluster'):
    count = len(data)
    withinClustSS = data.squaredDistance.sum()
    print(f'Cluster {cluster} ({count} members): {withinClustSS:.2f} within cluster ')


In [ ]:
# Euclidean Distance between Cluster centroids

pd.DataFrame(pairwise.pairwise_distances(kmeans.cluster_centers_, metric='euclidean'))


In [ ]:
# code for plotting profile plot of centroids

centroids['cluster'] = ['Cluster {}'.format(i) for i in centroids.index]
plt.figure(figsize=(10,6))
parallel_coordinates(centroids, class_column='cluster', colormap='Dark2', linewidth=5)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))



In [ ]:
# Plot ks
inertia = []
for n_clusters in range(1, 7):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(utilities_df_norm)
    inertia.append(kmeans.inertia_ / n_clusters)
inertias = pd.DataFrame({'n_clusters': range(1, 7), 'inertia': inertia})
ax = inertias.plot(x='n_clusters', y='inertia')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Average Within-Cluster Squared Distances')
plt.ylim((0, 1.1 * inertias.inertia.max()))
ax.legend().set_visible(False)
plt.show()


### Question 1.1
The variables that describe purchase behavior (including brand loyalty) (10 points)


---
**Answer:**



--- 
<br>

### Question 1.2
The variables that describe the basis for purchase (10 points)

---
**Answer:**



--- 
<br>

### Question 1.3
The variables that describe both purchase behavior and basis of purchase
(14 points)

**Note 1:** How should k be chosen? Think about how the clusters would be used. It is likely that the marketing efforts would support two to five different promotional approaches.

**Note 2:** How should the percentages of total purchases by various brands be treated? Isn’t a customer who buys all brand A just as loyal as a customer who buys all brand B? What will be the effect on any distance measure of using the brand share variables as is? Consider using a single derived variable.


---
**Answer:**



--- 
<br>

## Question 2
Select what you think is the best segmentation and comment on the characteristics (demographic, brand loyalty, and basis for purchase) of these clusters. (This information would be used to guide the development of advertising and promotional campaigns.)(14 points)


---
**Answer:**



--- 
<br>

## Question 3
Develop a model that classifies the data into these segments. Since this information would most likely be used in targeting direct-mail promotions, it would be useful to select a market segment that would be defined as a success in the classification model.(14 points)


---
**Answer:**



--- 
<br>

## Question 4
Briefly explain, in two to three paragraphs, the business objective, the data mining models used, why they were used, the model results, and your recommendations to your non-technical stakeholder team. (18 points)



---
**Answer:**



--- 
<br>